In [43]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled = [2005, 2021, False, True]
else:
    start_year, end_year, chk, sampled = [2005, 2021, True, False]

undersampling = True
oversampling  = False
smote         = False

In [47]:
from sklearn.model_selection import train_test_split
from my_libs.scalers import FeaturesScaler

df = pd.read_pickle("./project_df.pkl")
data = df.iloc[:, 1:]
target = df['grav']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)

In [48]:
from my_libs.encoders import FeaturesEncoder
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced

encoder = FeaturesEncoder(True)
scaler  = FeaturesScaler(True)
smt     = SMOTE(random_state=42)
model   = DecisionTreeClassifier(max_depth=10)

X_train = encoder.fit_transform(X_train, y_train)
X_train = scaler.fit_transform(X_train, y_train)
X_train, y_train = smt.fit_resample(X_train, y_train)
model.fit(X_train, y_train)

X_test = encoder.fit_transform(X_test, y_test)
X_test = scaler.fit_transform(X_test, y_test)
X_test, y_test = smt.fit_resample(X_test, y_test)
y_pred = model.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

Column jour has been target encoded
Column age has been target encoded
Column dep has been target encoded
Column place has been dummy encoded
Column catu has been dummy encoded
Column sexe has been dummy encoded
Column trajet has been dummy encoded
Column locp has been dummy encoded
Column actp has been dummy encoded
Column etatp has been dummy encoded
Column mois has been dummy encoded
Column lum has been dummy encoded
Column agg has been dummy encoded
Column int has been dummy encoded
Column atm has been dummy encoded
Column col has been dummy encoded
Column catr has been dummy encoded
Column circ has been dummy encoded
Column nbv has been dummy encoded
Column vosp has been dummy encoded
Column prof has been dummy encoded
Column plan has been dummy encoded
Column surf has been dummy encoded
Column infra has been dummy encoded
Column situ has been dummy encoded
Column senc has been dummy encoded
Column catv has been dummy encoded
Column joursem has been dummy encoded
Column jour has b

In [45]:
from my_libs.encoders import FeaturesEncoder
from sklearn.tree import DecisionTreeClassifier
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
# from sklearn.metrics import classification_report
from imblearn.metrics import classification_report_imbalanced

encoder = FeaturesEncoder()
scaler  = FeaturesScaler(True)
smt     = SMOTE(random_state=42)
model   = DecisionTreeClassifier()

dt_pipe = Pipeline([
                    ('var_encoding', encoder),
                    ('scaling', scaler),
                    ('smote', smt),
                    # ('selection', selector),
                    ('model', model)
                    ])
param_grid = {
    'model__max_depth' : (10, 20),
    # 'model__criterion' : ('gini', 'entropy'),
    # 'model__max_features' : ('auto', 'sqrt', 'log2'),
    # 'model__min_samples_split' : (2,4,6)
}

# dt_pipe.fit(X_train, y_train)

grid = GridSearchCV(dt_pipe, param_grid=param_grid, cv = 3)
grid.fit(X_train, y_train)
#
# print('Le meilleur score obtenu est: ',grid.best_score_)
# print('Les meilleurs paramètres trouvés sont :',grid.best_params_)
#
# y_pred = dt_pipe.predict(X_test)
# print(classification_report_imbalanced(y_test, y_pred))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\metaestimators.py", line 113, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py", line 707, in score
    Xt = transform.transform(Xt)
  File "C:\Users\nicol\git\projet-accidents-routiers\my_libs\encoders.py", line 19, in transform
    X_enc[f"{col}"] = TargetEncoder().fit_

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('var_encoding', FeaturesEncoder()),
                                       ('scaling',
                                        <my_libs.scalers.FeaturesScaler object at 0x00000184B2E415B0>),
                                       ('smote', SMOTE(random_state=42)),
                                       ('model', DecisionTreeClassifier())]),
             param_grid={'model__max_depth': (10, 20)})

In [ ]:
y_pred = dt_pipe.predict(X_test)
# print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
dt_pipe

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler, SMOTE
#
# if undersampling:
#     # Random Undersampling
#     rUs = RandomUnderSampler()
#     # X_ru, y_ru = rUs.fit_resample(X_train, y_train)
#     X_train, y_train = rUs.fit_resample(X_train, y_train)
#     # print('Classes échantillon undersampled :', y_ru.value_counts())
#     print('Classes échantillon undersampled :', y_train.value_counts())
# if oversampling:
#     rOs = RandomOverSampler()
#     X_train, y_train = rOs.fit_resample(X_train, y_train)
#     print('Classes échantillon oversampled :', dict(pd.Series(y_train).value_counts()))
# if smote:
#     smo = SMOTE()
#     X_train, y_train = smo.fit_resample(X_train, y_train)
#     print('Classes échantillon SMOTE :', dict(pd.Series(y_train).value_counts()))

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.tree import DecisionTreeClassifier
#
# parameters = {'max_depth'         : (10,20,30,40,50),
#               'criterion'         : ('gini', 'entropy'),
#               'max_features'      : ('auto', 'sqrt', 'log2'),
#               'min_samples_split' : (2,4,6)
#               }
#
# grid = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=parameters, cv=5, verbose=True)
#
# grid.fit(X_train, y_train)

In [ ]:
# from imblearn.metrics import classification_report_imbalanced
#
# model = grid.best_estimator_
# print(model)
#
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
#
# print(f"\n{classification_report_imbalanced(y_test, y_pred)}")


In [ ]:
# feats = {}
# for feature, importance in zip(data.columns, model.feature_importances_):
#     feats[feature] = importance
#
# importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
#
# #Affichage des 8 variables les plus importantes
# importances.sort_values(by='Gini-importance', ascending=False).head(8)

In [ ]:
# from sklearn import tree
# tree.plot_tree(model);


In [ ]:
# print(f"Train accuracy : {model.score(X_train, y_train):.3f}")
# print(f"Test accuracy : {model.score(X_test, y_test):.3f}")